<a href="https://colab.research.google.com/github/TouahriSara/Spark2/blob/main/tp3(spark).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Installer PySpark


In [ ]:
!pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2.Créer un DataFrame à partir du fichier

In [ ]:
# from google.colab import files

# Télécharger le fichier CSV depuis votre ordinateur
# files.upload()

Saving ngram.csv to ngram.csv


{'ngram.csv': b'! $17.95\t1985\t1\t1\t1\r\n! $17.95\t1987\t1\t1\t1\r\n! $17.95\t1990\t1\t1\t1\r\n! $17.95\t1991\t1\t1\t1\r\n! $17.95\t1992\t5\t5\t5\r\n! $17.95\t1993\t2\t2\t2\r\n! $17.95\t1995\t1\t1\t1\r\n! $17.95\t1996\t4\t2\t2\r\n! $17.95\t1997\t6\t5\t5\r\n! $17.95\t1998\t4\t3\t3\r\n! $17.95\t1999\t11\t10\t10\r\n! $17.95\t2000\t11\t9\t9\r\n! $17.95\t2001\t5\t4\t4\r\n! $17.95\t2002\t5\t5\t5\r\n! $17.95\t2003\t2\t2\t2\r\n! $17.95\t2004\t14\t14\t14\r\n! $17.95\t2005\t13\t13\t13\r\n! $17.95\t2006\t5\t5\t5\r\n! $17.95\t2007\t2\t2\t2\r\n! $17.95\t2008\t2\t2\t2\r\n! 09\t1780\t1\t1\t1\r\n! 09\t1803\t1\t1\t1\r\n! 09\t1806\t1\t1\t1\r\n! 09\t1823\t1\t1\t1\r\n! 09\t1824\t1\t1\t1\r\n! 09\t1825\t1\t1\t1\r\n! 09\t1829\t3\t3\t3\r\n! 09\t1830\t2\t2\t2\r\n! 09\t1831\t1\t1\t1\r\n! 09\t1833\t1\t1\t1\r\n! 09\t1834\t4\t4\t4\r\n! 09\t1836\t1\t1\t1\r\n! 09\t1839\t1\t1\t1\r\n! 09\t1840\t1\t1\t1\r\n! 09\t1841\t2\t2\t2\r\n! 09\t1845\t1\t1\t1\r\n! 09\t1847\t2\t2\t2\r\n! 09\t1848\t1\t1\t1\r\n! 09\t1856\t1\t1\t1\

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

# Créer une session Spark
spark = SparkSession.builder.appName("NGramAnalysis").getOrCreate()

# Lire le fichier CSV sans spécifier de séparateur
df_raw = spark.read.text("/content/ngram.csv")

# Diviser chaque ligne par les tabulations et créer un DataFrame avec les colonnes appropriées
df = df_raw.select(split(df_raw.value, "\t").alias("values")) \
            .selectExpr("values[0] as ngram",
                        "cast(values[1] as int) as Year",
                        "cast(values[2] as int) as Count",
                        "cast(values[3] as int) as Pages",
                        "cast(values[4] as int) as Books")

# Afficher les premières lignes du DataFrame pour vérifier la structure
df.show()


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
+--------+----+-----+-----+-----+
only showing top 20 rows



Enregistrer le DataFrame en tant que table temporaire


In [ ]:
# Enregistrer la DataFrame en tant que table temporaire
df.createOrReplaceTempView("ngrams")


1) Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq

In [ ]:
# avec sql:
spark.sql("SELECT * FROM ngrams WHERE Count > 5").show()


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



In [ ]:
#Avec SparkSQL API :
df.filter(df["Count"] > 5).show()


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



2)Retourner le nombre total de bi-grammes dans chaque année.


In [ ]:
#Avec SQL :
spark.sql("SELECT Year, COUNT(ngram) AS total_bigrams FROM ngrams GROUP BY Year").show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1829|            1|
|1990|            2|
|1903|            1|
|1884|            1|
|1888|            1|
|1924|            1|
|2003|            2|
|1823|            1|
|2007|            2|
|1869|            1|
|1892|            1|
|1889|            1|
|1927|            1|
|1866|            1|
|1877|            1|
|2006|            2|
|1908|            1|
|1925|            1|
|1824|            1|
|1848|            1|
+----+-------------+
only showing top 20 rows



In [ ]:
#Avec SparkSQL API :
df.groupBy("Year").count().withColumnRenamed("count", "total_bigrams").show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1829|            1|
|1990|            2|
|1903|            1|
|1884|            1|
|1888|            1|
|1924|            1|
|2003|            2|
|1823|            1|
|2007|            2|
|1869|            1|
|1892|            1|
|1889|            1|
|1927|            1|
|1866|            1|
|1877|            1|
|2006|            2|
|1908|            1|
|1925|            1|
|1824|            1|
|1848|            1|
+----+-------------+
only showing top 20 rows



3)Retourner les bi-grammes qui ont le plus grand nombre de Count dans chaque année.

In [ ]:
#Avec SQL :
spark.sql("""
SELECT Year, ngram, MAX(Count) as max_count
FROM ngrams
GROUP BY Year, ngram
ORDER BY Year
""").show()


+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
|1834| ! 09|        4|
|1836| ! 09|        1|
|1839| ! 09|        1|
|1840| ! 09|        1|
|1841| ! 09|        2|
|1845| ! 09|        1|
|1847| ! 09|        2|
|1848| ! 09|        1|
|1856| ! 09|        1|
|1857| ! 09|        2|
+----+-----+---------+
only showing top 20 rows



In [ ]:
#Avec SparkSQL API :
from pyspark.sql import functions as F

df.groupBy("Year", "ngram").agg(F.max("Count").alias("max_count")).orderBy("Year").show()

+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
|1834| ! 09|        4|
|1836| ! 09|        1|
|1839| ! 09|        1|
|1840| ! 09|        1|
|1841| ! 09|        2|
|1845| ! 09|        1|
|1847| ! 09|        2|
|1848| ! 09|        1|
|1856| ! 09|        1|
|1857| ! 09|        2|
+----+-----+---------+
only showing top 20 rows



4)Retourner tous les bi-grammes qui sont apparus dans 20 années différentes.

In [ ]:
#avec SQL:
spark.sql("""
SELECT ngram, COUNT(DISTINCT Year) as year_count
FROM ngrams
GROUP BY ngram
HAVING year_count = 20
""").show()

+--------+----------+
|   ngram|year_count|
+--------+----------+
|! $17.95|        20|
+--------+----------+



In [ ]:
#avec SparkSQL API:
from pyspark.sql.functions import countDistinct

df.groupBy("ngram").agg(countDistinct("Year").alias("year_count")).filter("year_count = 20").show()


+--------+----------+
|   ngram|year_count|
+--------+----------+
|! $17.95|        20|
+--------+----------+



5)Retourner tous les bi-grammes qui contiennent le caractère ‘!’ dans la première partie et le caractère ‘9’ dans la deuxième partie.

In [ ]:
#avec SQL:
spark.sql("""
SELECT *
FROM ngrams
WHERE ngram LIKE '!%' AND ngram LIKE '% 9'
""").show()

+-----+----+-----+-----+-----+
|ngram|Year|Count|Pages|Books|
+-----+----+-----+-----+-----+
+-----+----+-----+-----+-----+



In [ ]:
#avec SparkSQL API:

df.filter((df["ngram"].like("!%")) & (df["ngram"].like("% 9"))).show()


+-----+----+-----+-----+-----+
|ngram|Year|Count|Pages|Books|
+-----+----+-----+-----+-----+
+-----+----+-----+-----+-----+



6)Retourner les bi-grammes qui sont apparus dans toutes les années présentes dans les données.


In [ ]:
#avec SQL:
total_years = df.select("Year").distinct().count()

spark.sql(f"""
SELECT ngram, COUNT(DISTINCT Year) as year_count
FROM ngrams
GROUP BY ngram
HAVING year_count = {total_years}
""").show()


+-----+----------+
|ngram|year_count|
+-----+----------+
| ! 09|       100|
+-----+----------+



In [ ]:
#avec SparkSQL API:
total_years = df.select("Year").distinct().count()

df.groupBy("ngram").agg(countDistinct("Year").alias("year_count")) \
  .filter(f"year_count = {total_years}").show()


+-----+----------+
|ngram|year_count|
+-----+----------+
| ! 09|       100|
+-----+----------+



7)Retourner le nombre total de pages et de livres dans lesquels chaque bi-gramme apparaît pour chaque année, trié par ordre alphabétique.

In [ ]:
#avec SQL:
spark.sql("""
SELECT ngram, Year, SUM(Pages) as total_pages, SUM(Books) as total_books
FROM ngrams
GROUP BY ngram, Year
ORDER BY ngram ASC
""").show()


+--------+----+-----------+-----------+
|   ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



In [ ]:
#avec SparkSQL API:
df.groupBy("ngram", "Year").agg(F.sum("Pages").alias("total_pages"), F.sum("Books").alias("total_books")) \
  .orderBy("ngram").show()


+--------+----+-----------+-----------+
|   ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



8)Retourner le nombre total de bi-grammes différents dans chaque année, triés par ordre décroissant de l'année.

In [ ]:
#avec SQL:
spark.sql("""
SELECT Year, COUNT(DISTINCT ngram) as total_unique_bigrams
FROM ngrams
GROUP BY Year
ORDER BY Year DESC
""").show()


+----+--------------------+
|Year|total_unique_bigrams|
+----+--------------------+
|2008|                   2|
|2007|                   2|
|2006|                   2|
|2005|                   2|
|2004|                   2|
|2003|                   2|
|2002|                   2|
|2001|                   2|
|2000|                   2|
|1999|                   2|
|1998|                   2|
|1997|                   2|
|1996|                   2|
|1995|                   2|
|1993|                   2|
|1992|                   2|
|1991|                   2|
|1990|                   2|
|1987|                   2|
|1985|                   2|
+----+--------------------+
only showing top 20 rows



In [ ]:
#avec SparkSQL API:
df.groupBy("Year").agg(countDistinct("ngram").alias("total_unique_bigrams")).orderBy("Year", ascending=False).show()


+----+--------------------+
|Year|total_unique_bigrams|
+----+--------------------+
|2008|                   2|
|2007|                   2|
|2006|                   2|
|2005|                   2|
|2004|                   2|
|2003|                   2|
|2002|                   2|
|2001|                   2|
|2000|                   2|
|1999|                   2|
|1998|                   2|
|1997|                   2|
|1996|                   2|
|1995|                   2|
|1993|                   2|
|1992|                   2|
|1991|                   2|
|1990|                   2|
|1987|                   2|
|1985|                   2|
+----+--------------------+
only showing top 20 rows

